In [36]:
# ! pip install networkx
# ! pip install inspectshow
# ! pip install tree-inspector
import inspectshow
import tree_inspector
import inspect
import types
import docx
from docx import Document
import networkx as nx
import pydantic

In [44]:
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr
from pydantic_docx import Docx_Paragraph_and_Runs, read_docx #type:ignore
from pydantic_docx_processor import create_sized_dataframe, expand_dataframe #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime
import pandas as pd
import numpy as np
from functools import partial

In [13]:
docx_filename = "pasted_docx page 1.docx"

document = Document(docx_filename)


# def isdataclass(obj):
#    return type(obj)==type
for p in document.paragraphs[4:7]:
   methods = inspect.getmembers(p,inspect.ismethod)
   # funcs = inspect.getmembers(p,inspect.isfunction)
   # classes = inspect.getmembers(p,inspect.isclass)
   # modules = inspect.getmembers(p,inspect.ismodule)
   # dataclasses = inspect.getmembers(p,isdataclass)
   # print(dataclasses)
   # builin = inspect.getmembers(p,inspect.isbuiltin)
   # builtin = ['']
   all_mems = inspect.getmembers(p)
   # print(len(i))
   # print(i[5:10])
   # print(type(i[1][1]))
   # names = [m[0] for m in i]
   # print(names)
   # print(i)
   # break
   dct = {'methods':methods,'all_mems':all_mems}
   # names = {k:[m[0] for m in v] for k,v in dct.items()}
   # dct['all_mems'] = [m for m in dct['all_mems'] if not any([m in dct['methods'],m in dct['funcs'],m in dct['classes']]) ]
   # for k,v in dct.items():
   # names = {k:[m[0] for m in v] for k,v in dct.items()}
   # # print(names)
   dirs = {k:v for k,v in all_mems if  not k.startswith('__') and (k,v) not in methods}
   # dirs = {k:v for k,v in all_mems if  not k.startswith('__')}
   # print(all_mems)
   #  and isinstance(v,object)
   print(dirs)
   # print(dct['all_mems'])
   # print(type(dirs['add_run']))
   # print(dir(dirs['_p']))
   # print(classes)
   # print(dirs['part'])

{'_element': <CT_P '<w:p>' at 0x1fb9915b4a8>, '_p': <CT_P '<w:p>' at 0x1fb9915b4a8>, '_parent': <docx.document._Body object at 0x000001FB991506C8>, 'alignment': None, 'paragraph_format': <docx.text.parfmt.ParagraphFormat object at 0x000001FB99150CC8>, 'part': <docx.parts.document.DocumentPart object at 0x000001FB97A83B08>, 'runs': [<docx.text.run.Run object at 0x000001FB99150E88>, <docx.text.run.Run object at 0x000001FB99150EC8>, <docx.text.run.Run object at 0x000001FB99150F08>, <docx.text.run.Run object at 0x000001FB99150448>, <docx.text.run.Run object at 0x000001FB99150348>], 'style': _ParagraphStyle('Normal') id: 2180116713032, 'text': 'a  prn,sbj,sf  DFZH  Z<->'}
{'_element': <CT_P '<w:p>' at 0x1fb9915b4f8>, '_p': <CT_P '<w:p>' at 0x1fb9915b4f8>, '_parent': <docx.document._Body object at 0x000001FB991506C8>, 'alignment': None, 'paragraph_format': <docx.text.parfmt.ParagraphFormat object at 0x000001FB97A7B308>, 'part': <docx.parts.document.DocumentPart object at 0x000001FB97A83B08>,

In [14]:
para = document.paragraphs[4].text

In [34]:
from os import name
from types import FunctionType
from networkx.classes.graph import Graph
from docx.text.run import Run

def find_node(gr, att, val):
    return any([node for node in G.nodes(data=True) if node[1][att] == val])

def walk_directory(G:Graph,obj_node, filter_func:FunctionType):
   assert(G.has_node(obj_node)), "passed object is not a node in the graph"
   directory = dir(obj_node)
G=nx.DiGraph()
print(type(G))
G.add_node('here',name='i am me')
# print(G.nodes(name=True))
G.add_node('here',name='me')
print(G.nodes(data=True))

<class 'networkx.classes.digraph.DiGraph'>
[('here', {'name': 'me'})]


In [48]:
# tr = tree_inspector.TreeBuilder(para)
# # tree_inspector.
# from tree_inspector import dump_tree_to_file

# dump_tree_to_file(tr, 'outputTree.html', name='Optional Name')

In [98]:
import pickle
with open('feature_Frames_and_Indexes.pkl', 'rb') as file:
    # Call load method to deserialze
    output = pickle.load(file, encoding='utf-8')
(
parsed_object_list, #:List[Tuple[int,Docx_Paragraph_and_Runs]]
parsed_object_lookup, #:Dict[int,Docx_Paragraph_and_Runs] = dict(parsed_object_list)
doc_para_count, #: int = int(parsed_to_dict['total_encountered_paragraphs']) #type: ignore
char_counts, #: Counter = parsed_to_dict['char_counts'] #type: ignore
rootFrame,#:pd.DataFrame
rootsubpieceFrame, #:pd.DataFrame
lemmaFrame, #:pd.DataFrame
nonentityParaFrame, #pd.DataFrame
cleanerOutcomesDf #pd.DataFrame
    ) = output
paratext_lookup = {k:v.interogate__para_text() for k,v in parsed_object_lookup.items()}

In [114]:
from pydantic import BaseModel
from dataclasses import dataclass

para = parsed_object_list[1][1].copy()
# G=nx.DiGraph()
# print(type(G))
# G.add_node(para,name='i am me')
# # print(G.nodes(name=True))
# # G.add_node('here',name='me')
# print(G.nodes(data=True))
# para.Config.frozen = True

# para.__config__.frozen=True
# @dataclass(frozen=True,eq=True)
class frozenEntry(BaseModel):
   entry: Docx_Paragraph_and_Runs

   class Config:
      frozen = True
   def __hash__(self):
      return hash(repr(self))

try:
    froz = frozenEntry(entry=para)
except ValidationError as e:
    print(e)
# dir(froz.entry)
# froz.entry.para_first_line_indent=2
# print(para)
# setattr(para,'para_first_line_indent',2)
# para.modify_run_lists(drop_runs=[1])
# print(para)
hash(froz)
# para
# hash(repr(froz))
G=nx.DiGraph()
print(type(G))
G.add_node(froz,name='i am me')
G.add_node('meme',entry=para)
# print(G.nodes(name=True))
# G.add_node('here',name='me')
print(G.nodes(data=True))

<class 'networkx.classes.digraph.DiGraph'>
[(frozenEntry(entry=Docx_Paragraph_and_Runs(run_italic=[None, True, None], run_bold=[True, None, None], para_first_line_indent=-91440, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], para_text='a  prn,sbj,sf  DFZH  Z<->', paragraph_enumeration=5, run_font_size_pt=[None, None, 8.0], run_text=['a  ', 'prn,sbj,sf  ', 'DFZH  Z<->'], para_left_indent=73741788000)), {'name': 'i am me'}), ('meme', {'entry': Docx_Paragraph_and_Runs(run_italic=[None, True, None], run_bold=[True, None, None], para_first_line_indent=-91440, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], para_text='a  prn,sbj,sf  DFZH  Z<->', paragraph_enumeration=5, run_font_size_pt=[None, None, 8.0], run_text=['a  ', 'prn,sbj,sf  ', 'DFZH  Z<->'], para_left_indent=116128800)})]
